<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [1]:
from google.colab import drive
drive.mount('/content/drive') # , force_remount=True

address = "University/Term 6/MIR/Homeworks/HW3"
import sys
sys.path.append(f"/content/drive/My Drive/{address}")

%cd /content/drive/My\ Drive/$address

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/University/Term 6/MIR/Homeworks/HW3


In [2]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
!pip install hazm
!pip install unidecode
# !pip install pandas
# !pip install nltk
# !pip install torch
# !pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import faiss
import pickle
import zipfile
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from bs4 import BeautifulSoup
from string import punctuation
from transformers import pipeline
from IPython.display import display
from sentence_transformers import SentenceTransformer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٨٠ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [4]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [ ]:
class Scraper:

    def __init__(self, current_year, current_month):
        self.current_year = current_year
        self.current_month = current_month

    def get_URL_content(self, URL):
        while True:
            try:
                return requests.get(URL, timeout=5).content
            except:
                pass

    def generate_page_URL(self, page_index, category, year, month):
        tp = {'Politics': 6, 'World': 11, 'Economy': 10, 'Society': 5, 'City': 7,
              'Sport': 9, 'Science': 20, 'Culture': 26, 'IT': 718, 'LifeSkills': 21}[category]
        return f'https://www.hamshahrionline.ir/archive?pi={page_index}&tp={tp}&ty=1&ms=0&mn={month}&yr={year}'

    def get_page_URLs_by_time(self, category, year, month):
        URLs = []
        page_index = 1
        while True:
            URL = self.generate_page_URL(page_index, category, year, month)
            content = self.get_URL_content(URL)
            if re.findall('pagination', str(content)):
                URLs.append(URL)
                page_index += 1
            else:
                break
        return URLs

    def get_page_URLs_since(self, category, year, month):
        URLs = []
        with tqdm() as pbar:
            while True:
                if month > 12:
                    month = 1
                    year += 1
                pbar.set_description(f'[{category}] [Extracting page URLs] [Date: {year}/{month}]')
                URLs_by_time = self.get_page_URLs_by_time(category, year, month)
                if URLs_by_time:
                    for URL in URLs_by_time:
                        URLs.append(URL)
                    month += 1
                elif self.current_year > year or (self.current_year == year and self.current_month > month):
                    month += 1
                else:
                    break
        return URLs

    def get_news_URLs_since(self, category, year, month):
        news_URLs = []
        page_URLs = self.get_page_URLs_since(category, year, month)
        with tqdm(page_URLs) as pbar:
            for page_URL in pbar:
                content = self.get_URL_content(page_URL)
                soup = BeautifulSoup(content, 'html5lib')
                for item in soup.findAll('li', attrs={'class': 'news'}):
                    URL = item.find('div', attrs={'class': 'desc'}).find('h3').find('a')['href']
                    URL = 'https://www.hamshahrionline.ir' + URL
                    news_URLs.append(URL)
                pbar.set_description(f'[{category}] [Extracting news URLs] [{len(news_URLs)} news until now]')
        return news_URLs

    def parse_news(self, URL, category):
        try:
            content = self.get_URL_content(URL)
            soup = BeautifulSoup(content, 'html.parser')
            date = soup.find('div', {'class': 'col-6 col-sm-4 col-xl-4 item-date'}).span.text.strip()
            title = soup.find('div', {'class': 'item-title'}).h1.text.strip()
            intro = soup.find('p', {'class': 'introtext', 'itemprop': 'description'}).text.strip()
            body = soup.find('div', {'class': 'item-text', 'itemprop': 'articleBody'}).text.strip()
            category_PER = soup.find_all('li', {'class': 'breadcrumb-item'})
            category_PER = list(map(lambda x: x.text.strip(), category_PER))[1:]
            return {
                'date': date,
                'title': title,
                'intro': intro,
                'body': body,
                'category-PER': category_PER,
                'category-ENG': category,
            }
        except:
            return None

    def scrape(self, from_year, from_month):
        categories = ['Politics', 'World', 'Economy', 'Society', 'City',
                      'Sport', 'Science', 'Culture', 'IT', 'LifeSkills']
        news = []
        for category in categories:
            URLs = self.get_news_URLs_since(category, from_year, from_month)
            with tqdm(URLs) as pbar:
                pbar.set_description(f'[{category}] [Scraping news]')
                for URL in pbar:
                    news.append(self.parse_news(URL, category))
        news = list(filter(None, news))
        pd.DataFrame(news).to_csv(f'dataset.csv', encoding='utf-8')
        with zipfile.ZipFile('dataset.zip', 'w', zipfile.ZIP_DEFLATED) as zip_file:
            zip_file.write('dataset.csv')
        os.remove('dataset.csv')

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Scraper ایجاد شده و شروع به دریافت و ذخیره‌سازی داده‌ها می‌کند. خبرهای دریافت شده همگی مربوط به قرن جدید، از سال ١٤٠٠ به بعد هستند.
    </p>
</div>

In [ ]:
scraper = Scraper(current_year=1401, current_month=3)
# scraper.scrape(from_year=1400, from_month=1)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        بعد از ذخیره شدن داده‌ها در فایل فشرده dataset.zip، آن‌ها را از این فایل استخراج کرده و وارد برنامه می‌کنیم. با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [9]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [7]:
def display_dataset_info():
    global CATEGORIES

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for data in dataset:
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

,دسته,تعداد
0,کل خبرها,68362
1,سیاسی,15798
2,جهانی,2895
3,اقتصادی,8900
4,اجتماعی,13585
5,شهری,3853
6,ورزشی,8348
7,علمی,3190
8,فرهنگی,6512
9,فناوری اطلاعات,437


<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. در ادامه، به توضیح هر یک از مراحل پیش‌پردازش می‌پردازیم.
    </p>
</div>

In [10]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]


def save_preprocessed_texts(texts, path="preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)

In [11]:
def data_to_text(data):
    return ' '.join([data['title'], data['intro'], data['body']]).lower()


preprocessor = Preprocessor(stopwords_path='stopwords.txt')

texts = [data_to_text(data) for _, data in dataset.iterrows()]
preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
save_preprocessed_texts(preprocessed_texts)

# preprocessed_texts = load_preprocessed_texts()

100%|██████████| 68362/68362 [06:13<00:00, 183.19it/s]


In [12]:
dataset

,date,title,intro,body,category
0,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۴۰,واشنگتن: آمریکا و ایران هدف مشترکی دارند,سخنگوی وزارت خارجه آمریکا شامگاه سه‌شنبه در کن...,به گزارش همشهری‌آنلاین به نقل از فارس، ند پرای...,Politics
1,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۳۱,عراقچی باز هم ادعاهای «منبع مطلع» پرس‌تی‌وی در...,معاون وزیر امور خارجه جمهوری اسلامی ایران در ت...,به گزارش همشهری‌آنلاین به نقل از ایرنا، عباس ع...,Politics
2,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۳:۲۰,هشدار به دیپلمات‌های آمریکا در چاد؛ مراقب جان ...,وزارت خارجه آمریکا با صدور بیانیه‌ای به تمام د...,به گزارش همشهری آنلاین به نقل از فارس، به دنبا...,Politics
3,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۷,نامه مشاور رهبری به نمکی درباره واکسن پولی,از درگیر کردن بخش خصوصی به شدت پرهیز شود، چه د...,به گزارش همشهری آنلاین به نقل از ایسنا، در نام...,Politics
4,سه‌شنبه ۳۱ فروردین ۱۴۰۰ - ۲۲:۴۵,درخواست از روحانی: کشور را قرنطینه کامل کنید,حزب اتحاد ملت در نامه‌ای خطاب به روحانی خواستا...,به گزارش همشهری آنلاین به نقل از خبرآنلاین، مت...,Politics
...,...,...,...,...,...
68357,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۵۴,از زینک چه می‌دانید؟ | علائم کمبود زینک یا روی,بدن شما نیاز به انواع مواد مغذی دارد تا عملکرد...,به گزارش همشهری آنلاین، بدن شما نیاز به انواع ...,LifeSkills
68358,یکشنبه ۱ خرداد ۱۴۰۱ - ۱۱:۰۲,یک راهکار ساده برای مطمئن شدن از کیفیت عینک آف...,استفاده از عینک آفتابی یک کار ضروری برای محافظ...,به گزارش همشهری آنلاین، بررسی‌ها نشان داده قرا...,LifeSkills
68359,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۸:۰۰,تصاویری از سرزمین لک‌لک‌های ایران | زریوار میز...,بهار فصل خوبی برای سفر به کردستان است؛ جاده‌ها...,به گزارش همشهری آنلاین، شنیدن صدای لک‌لک‌ها یک...,LifeSkills
68360,یکشنبه ۱ خرداد ۱۴۰۱ - ۰۷:۵۵,۱۰ ترفند خانگی که سردردتان را بدون نیاز به قرص...,سردرد مشکل نادری نیست و همه‌ ما گاهی دچارش می‌...,به گزارش همشهری آنلاین به نقل از سلامت نیوز، ت...,LifeSkills


In [32]:
class Transformer:

    def __init__(self, preprocessor=None):
        self.model = SentenceTransformer('HooshvareLab/bert-fa-zwnj-base')
        self.preprocessor = preprocessor
        if torch.cuda.is_available():
            self.model = self.model.to(torch.device("cuda"))
        self.embeddings = None
        self.index = None
    
    def train_embeddings(self, train_dataset: list):
        if type(train_dataset[0]) == list:
            train_dataset = list(map(lambda doc: ' '.join(doc), train_dataset))
        self.embeddings = self.model.encode(train_dataset, show_progress_bar=True)
        self.embeddings = np.array(
            [embedding for embedding in self.embeddings]).astype("float32")
    
    def make_index(self, dataset: list):
        self.index = faiss.IndexFlatL2(self.embeddings.shape[1])
        self.index = faiss.IndexIDMap(self.index)
        self.index.add_with_ids(self.embeddings, np.array(list(range(len(dataset)))))

    def save_embeddings(self, path):
        with open(path, "wb") as f:
            pickle.dump(self.embeddings, f)
        
    def load_embeddings(self, path):
        with open(path, "rb") as f:
            self.embeddings = pickle.load(f)
    
    def predict(self, query: str, dataset: pd.DataFrame, k=10):
        if self.preprocessor:
            query = ' '.join(self.preprocessor.preprocess(query))
        vector = self.model.encode(list([query]))
        D, I = self.index.search(np.array(vector).astype("float32"), k=k)
        return [dataset.iloc[i] for i in I.flatten().tolist()]
        

In [33]:
transformer = Transformer(preprocessor)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab

In [34]:
transformer.train_embeddings(dataset.title.to_list())

Batches:   0%|          | 0/2137 [00:00<?, ?it/s]

In [35]:
transformer.save_embeddings("title_embeddings")

In [36]:
transformer.load_embeddings("title_embeddings")

In [37]:
transformer.make_index(dataset.title.to_list())

In [38]:
pd.DataFrame(transformer.predict("فدراسیون فوتبال", dataset))

,date,title,intro,body,category
28924,جمعه ۱۷ اردیبهشت ۱۴۰۰ - ۱۰:۴۴,تشکیل تیم فوتبال بهبودیافتگان,اداره ورزش شهرداری منطقه ۱۶ قصد دارد از میان ب...,مدیر اداره ورزش شهرداری منطقه ۱۶ با بیان اینکه...,Society
66713,پنجشنبه ۲۳ دی ۱۴۰۰ - ۰۱:۳۲,عکس روز| فوتبال قطع عضو,بازیکنان فوتبال قطع عضو در یک مسابقه مقدماتی د...,فوتبال قطع عضو گونه‌ای از فوتبال است که برای ا...,LifeSkills
51404,پنجشنبه ۷ بهمن ۱۴۰۰ - ۱۷:۵۳,ویدیو | صحبت های رئیس فدراسیون فوتبال در ورزشگ...,رئیس فدراسیون فوتبال در بدو ورود به ورزشگاه آز...,به گزارش همشهری آنلاین، شهاب الدین عزیزی خادم ...,Sport
48243,پنجشنبه ۱۸ شهریور ۱۴۰۰ - ۱۶:۲۶,قول رئیس فیفا به هواداران فوتبال ایران و عزیزی...,رئیس فدراسیون جهانی فوتبال برای هواداران در ای...,به گزارش همشهری‌آنلاین، جیوانی اینفانتینو، رئی...,Sport
48980,سه‌شنبه ۶ مهر ۱۴۰۰ - ۱۰:۳۲,محکومیت باشگاه استقلال در کمیته تعیین وضعیت فد...,باشگاه استقلال در دو پرونده از سوی کمیته تعیین...,به گزارش همشهری‌آنلاین به نقل از سایت فدراسیون...,Sport
46967,شنبه ۱۹ تیر ۱۴۰۰ - ۱۲:۲۴,عکس | تجمع هواداران ذوب‌آهن مقابل فدراسیون فوتبال,تعدادی از هواداران تیم فوتبال ذوب‌آهن اصفهان م...,به گزارش همشهری‌آنلاین به نقل از فارس، جمعی از...,Sport
49684,چهارشنبه ۲۴ آذر ۱۴۰۰ - ۱۵:۲۷,آشنایی با سازمان لیگ فوتبال ایران,سازمان لیگ فوتبال ایران در ۱۱ آبان ۱۳۸۰ شکل گر...,به گزارش همشهری آنلاین، سازمان لیگ فوتبال ایر...,Sport
50902,پنجشنبه ۲۸ بهمن ۱۴۰۰ - ۱۷:۳۶,سرپرست فدراسیون فوتبال نیامده تغییر کرد,در حالی که صبح امروز سرپرست فدراسیون فوتبال تع...,به گزارش همشهری آنلاین پس از عزل موقت شهاب‌الد...,Sport
45829,شنبه ۱۸ اردیبهشت ۱۴۰۰ - ۱۲:۰۴,عکس | نماز جماعت رقیب استقلال در زمین فوتبال,اقدام عبادی مربی و بازیکنان الهلال پیش از دیدا...,به گزارش همشهری‌آنلاین، تیم فوتبال الهلال عربس...,Sport
49259,سه‌شنبه ۱۸ آبان ۱۴۰۰ - ۱۹:۰۵,فدراسیون فوتبال ایران در خطر تعلیق,در صورتی که اساسنامه فدراسیون تا اواسط اسفندما...,به گزارش همشهری آنلاین به نقل از ورزش سه، بحث ...,Sport


In [39]:
title_transformer = transformer

In [40]:
transformer = Transformer(preprocessor=preprocessor)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab_bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /root/.cache/torch/sentence_transformers/HooshvareLab

In [41]:
transformer.train_embeddings(preprocessed_texts)

Batches:   0%|          | 0/2137 [00:00<?, ?it/s]

In [42]:
transformer.save_embeddings("processed_embeddings")

In [46]:
transformer.load_embeddings("processed_embeddings")

In [47]:
transformer.make_index(preprocessed_texts)

In [48]:
pd.DataFrame(transformer.predict("پیروزی ایران در مقابل امارات", dataset))

,date,title,intro,body,category
46254,سه‌شنبه ۲۵ خرداد ۱۴۰۰ - ۲۱:۵۶,پایان نیمه اول ایران-عراق | برتری ایران با گل ...,سردار ازمون همانند بازی با بحرین برابر عراق هم...,به گزارش همشهری آنلاین تیم دیدار تیم‌های ملی ...,Sport
29991,سه‌شنبه ۱۸ خرداد ۱۴۰۰ - ۰۰:۱۶,تصاویر | ۶ قاب از برد شیرین ایران در برابر بحرین,تیم ملی فوتبال امشب در دومین بازی دور برگشت مر...,,Society
49229,پنجشنبه ۲۰ آبان ۱۴۰۰ - ۱۶:۳۸,پیروزی کره برابر امارات | چشم بادامی ها در تعق...,شاگردان بنتو با پیروزی برابر امارات ۱۱ امتیازی...,به گزارش همشهری آنلاین دیدار دو تیم کره جنوبی ...,Sport
48840,جمعه ۱۶ مهر ۱۴۰۰ - ۰۰:۲۵,عذرخواهی سردار آزمون از مردم ایران,مهاجم تیم ملی فوتبال کشورمان با انتشار متنی در...,به گزارش همشهری آنلاین به نقل از ایرنا، تیم مل...,Sport
48835,جمعه ۱۶ مهر ۱۴۰۰ - ۰۸:۲۶,واکنش ستاره سرشناس سابق هلند به پیروزی ایران م...,کلارنس سیدورف، ستاره سابق تیم ملی هلند، میلان ...,به گزارش همشهری‌آنلاین، دیدار تیم‌های ملی فوتب...,Sport
51268,سه‌شنبه ۱۲ بهمن ۱۴۰۰ - ۲۰:۰۵,ویدیو | گل ایران به امارات | درخشش ادامه دار ط...,تیم ملی فوتبال ایران موفق شد یک برصفر امارات ر...,به گزارش همشهری آنلاین، مهدی طارمی که در بازی ...,Sport
47894,دوشنبه ۴ مرداد ۱۴۰۰ - ۱۸:۰۸,زمان دیدار پرسپولیس و استقلال در لیگ قهرمانان ...,پرسپولیس در مرحله یک هشتم باید به مصاف رقیب تا...,به گزارش همشهری آنلاین دیدار تیم‌های فوتبال پر...,Sport
48844,پنجشنبه ۱۵ مهر ۱۴۰۰ - ۲۱:۵۰,اتفاق کم‌سابقه در بازی ایران و امارات | شجاع خ...,در دقیقه ۶۰ بازی ایران و امارات در حالیکه شجاع...,به گزارش همشهری آنلاین پس از اخراج شجاع خلیل ز...,Sport
50570,دوشنبه ۱۳ دی ۱۴۰۰ - ۱۸:۰۲,ویدیو | ۲ گل نیمه اول بازی تراکتور و پرسپولیس,تیم های پرسپولیس و تراکتور در هفته سیزدهم لیگ ...,به گزارش همشهری آنلاین، در این دیدار عیسی آل ک...,Sport
51031,جمعه ۲۲ بهمن ۱۴۰۰ - ۲۲:۰۷,ببینید | فرار الاهلی از شکست با درخشش ۲ ستاره ...,۲ بازیکن ایرانی الاهلی را از شکست نجات دادند.,به گزارش همشهری آنلاین در ادامه هفته پانزدهم ل...,Sport
